In [1]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import datasets
from datasets import load_dataset

In [3]:
dataset = load_dataset("tverous/anli-amr", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/991 [00:00<?, ?B/s]

(…)-00000-of-00001-99aac9afd7e04376.parquet:   0%|          | 0.00/42.8M [00:00<?, ?B/s]

(…)-00000-of-00001-a639eb6d952016f3.parquet:   0%|          | 0.00/847k [00:00<?, ?B/s]

(…)-00000-of-00001-9eb6c125c93e351c.parquet:   0%|          | 0.00/851k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100459 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [4]:
dataset[0]

{'uid': '2093cfb3-a15f-4282-81e3-0cb793ffd0d7',
 'premise': 'TOKYO, Dec 18 (Reuters) - Japan’s Shionogi & Co said on Tuesday that it has applied to health regulators in the United States, Canada and Europe for approval of its HIV drug Dolutegravir. Shionogi developed Dolutegravir with a Viiv Healthcare, an AIDS drug joint venture between GlaxoSmithKline and Pfizer, in exchange for its rights to the drug.',
 'hypothesis': 'The article was written on December 18th.',
 'label': 0,
 'reason': 'TOKYO, Dec 18 (Reuters) is when the article was written as it states in the first words of the sentence',
 'claim_cleaned_amr': '( z0 write :ARG1 ( z1 article ) :time ( z2 date-entity :day 18 :month 12 ) )',
 'amr_penman': '(z0 / write-01\n    :ARG1 (z1 / article)\n    :time (z2 / date-entity\n              :day 18\n              :month 12))',
 'amr_tokens': ['The',
  'article',
  'was',
  'written',
  'on',
  'December',
  '18th',
  '.'],
 'amr_nodes': "{'z1': 'article', 'z0': 'write-01', 'z2': 'dat

In [5]:
dataset.shape

(100459, 11)

In [6]:
dataset.column_names

['uid',
 'premise',
 'hypothesis',
 'label',
 'reason',
 'claim_cleaned_amr',
 'amr_penman',
 'amr_tokens',
 'amr_nodes',
 'amr_alignments',
 'amr_edges']

In [7]:
#Extract amr graphs with text for paired training

def extract_amr_and_text(data):
  amr_text_pairs = []
  for row in data:
    amr_graph = row.get("amr_penman", None)
    text = row.get("hypothesis", None)
    if amr_graph and text:
      amr_text_pairs.append({"amr_graph": amr_graph,"text": text})
  return amr_text_pairs

In [8]:
amr_text_pairs = extract_amr_and_text(dataset)

In [9]:
amr_text_pairs[0]

{'amr_graph': '(z0 / write-01\n    :ARG1 (z1 / article)\n    :time (z2 / date-entity\n              :day 18\n              :month 12))',
 'text': 'The article was written on December 18th.'}

In [10]:
amrs = []
text = []

for i in range(len(dataset)):
  amrs.append(amr_text_pairs[i]["amr_graph"])
  text.append(amr_text_pairs[i]["text"])

In [11]:
#Create dataframe with amr_graph and text columns

import pandas as pd

df = pd.DataFrame({"amr_graph": amrs, "text": text})

In [12]:
df.head()

,amr_graph,text
0,(z0 / write-01\n :ARG1 (z1 / article)\n ...,The article was written on December 18th.
1,(z0 / urge-01\n :ARG0 (z1 / person\n ...,Gillum was on TV urging residents to stay out ...
2,(z0 / and\n :op1 (z1 / beat-03\n ...,Carlton beat Melbourne in 2016 and will attemp...
3,(z0 / close-01\n :ARG1 (z1 / road)\n :du...,The road was closed for more than two hours af...
4,(z0 / advise-01\n :ARG2 (z1 / slow-down-03)),Its advisible to slow down


In [13]:
import torch
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, Trainer, AutoModelForCausalLM
import re
import pandas as pd

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [14]:
#Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [15]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [16]:
small_dataset = dataset.select([i for i in range(25000)])

In [17]:
#Define prompt and answer templates
prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction: {instruction}\n Response:"""
answer_template = """{response}"""

In [18]:
#Define function to add keys in the dictionary for prompt, answer and combined text
def _add_text(rec):
  instruction = rec["amr_graph"]
  response = rec["text"]

  #Check if both exist; raise error if not
  if not instruction:
    raise ValueError("instruction is missing")
  if not response:
    raise ValueError("Expected a response")

  #Create prompt, answer, combined text
  rec["prompt"] = prompt_template.format(instruction=instruction)
  rec["answer"] = answer_template.format(response=response)
  rec["text"] = rec["prompt"] + rec["answer"]
  return rec

In [19]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [20]:
small_dataset = small_dataset.map(_add_text)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [21]:
small_dataset[1]

{'amr_graph': '(z0 / urge-01\n    :ARG0 (z1 / person\n              :name (z2 / name\n                        :op1 "Gillum")\n              :medium (z3 / television))\n    :ARG1 (z4 / person\n              :ARG0-of (z5 / reside-01))\n    :ARG2 (z6 / stay-01\n              :ARG1 z4\n              :ARG3 (z7 / out-06\n                        :ARG1 z4\n                        :ARG2 (z8 / storm))))',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction: (z0 / urge-01\n    :ARG0 (z1 / person\n              :name (z2 / name\n                        :op1 "Gillum")\n              :medium (z3 / television))\n    :ARG1 (z4 / person\n              :ARG0-of (z5 / reside-01))\n    :ARG2 (z6 / stay-01\n              :ARG1 z4\n              :ARG3 (z7 / out-06\n                        :ARG1 z4\n                        :ARG2 (z8 / storm))))\n Response:Gillum was on TV urging residents to stay out of the storm.',
 'prompt': 'Below

In [22]:
tokenizer.pad_token = tokenizer.eos_token

In [23]:
MAX_LENGTH = 256

In [24]:
from typing import Dict, List
import copy
def preprocess_batch(batch: Dict[str,List]):
  model_inputs = tokenizer(batch["text"], padding='max_length', truncation=True, max_length=MAX_LENGTH)
  model_inputs["labels"] = copy.deepcopy(model_inputs["input_ids"])
  return model_inputs

In [25]:
from functools import partial
preprocessing_function = partial(preprocess_batch)

In [26]:
#Define split ratios

train_test_split = small_dataset.train_test_split(test_size=0.2) #20% as test
train_valid_split = train_test_split["train"].train_test_split(test_size=0.1) #From train, split 10% as validation

In [27]:
from datasets import DatasetDict
dataset_dict = DatasetDict({
    "train": train_valid_split["train"],
    "validation": train_valid_split["test"],
    "test": train_test_split["test"]
})

In [28]:
print(f"Train set size: {len(dataset_dict['train'])}")
print(f"Validation set size: {len(dataset_dict['validation'])}")
print(f"Test set size: {len(dataset_dict['test'])}")


Train set size: 18000
Validation set size: 2000
Test set size: 5000


In [29]:
# Example check for first item in each split
print("Sample from train:", dataset_dict['train'][0])
print("Sample from validation:", dataset_dict['validation'][0])
print("Sample from test:", dataset_dict['test'][0])

Sample from train: {'amr_graph': '(z0 / act-02\n    :ARG0 (z1 / person\n              :name (z2 / name\n                        :op1 "Rahal"))\n    :time (z3 / no-longer))', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction: (z0 / act-02\n    :ARG0 (z1 / person\n              :name (z2 / name\n                        :op1 "Rahal"))\n    :time (z3 / no-longer))\n Response:Rahal is no longer acting.', 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction: (z0 / act-02\n    :ARG0 (z1 / person\n              :name (z2 / name\n                        :op1 "Rahal"))\n    :time (z3 / no-longer))\n Response:', 'answer': 'Rahal is no longer acting.'}
Sample from validation: {'amr_graph': '(z0 / possible-01\n    :ARG1 (z1 / and\n              :op1 (z2 / act-01\n                       :ARG0 (z3 / person))\n              :op2 (z4 / run-02\n        

In [30]:
# Apply the preprocessing function to each batch in the dataset
encoded_train_dataset = dataset_dict['train'].map(
    preprocessing_function,
    batched=True,
    remove_columns=["amr_graph", "text", "prompt", "answer"],
)

encoded_validation_dataset = dataset_dict['validation'].map(
    preprocessing_function,
    batched=True,
    remove_columns=["amr_graph", "text", "prompt", "answer"],
)

encoded_test_dataset = dataset_dict['test'].map(
    preprocessing_function,
    batched=True,
    remove_columns=["amr_graph", "text", "prompt", "answer"],
)
processed_train_dataset = encoded_train_dataset.filter(lambda rec: len(rec["input_ids"]) <= MAX_LENGTH)
processed_validation_dataset = encoded_validation_dataset.filter(lambda rec: len(rec["input_ids"]) <= MAX_LENGTH)
processed_test_dataset = encoded_test_dataset.filter(lambda rec: len(rec["input_ids"]) <= MAX_LENGTH)


Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/18000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [31]:
processed_test_dataset.shape

(5000, 3)

In [37]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='/mnt/disks/disk1/results',  ## give the directory name where you want to save the model
    evaluation_strategy='epoch',
    report_to="none",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,  # Accumulate gradients for 4 steps
    warmup_steps=50,
    learning_rate=5e-5,        # Lowered learning rate
    weight_decay=0.1,          # Reduced weight decay to prevent over-penalizing weights
    logging_dir='/mnt/disks/disk1/logs' ## give the directory name where you want to save the model
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [38]:
from transformers import DataCollatorForLanguageModeling
# Initialize the data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [39]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_validation_dataset,
    data_collator=data_collator
)

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.366000,0.329752
2,1.194200,0.309852
3,1.062500,0.306736
4,0.991600,0.298060
5,0.927400,0.298291


TrainOutput(global_step=22500, training_loss=1.1502192518446182, metrics={'train_runtime': 3059.0868, 'train_samples_per_second': 29.421, 'train_steps_per_second': 7.355, 'total_flos': 1.175814144e+16, 'train_loss': 1.1502192518446182, 'epoch': 5.0})

In [41]:
# Save the model and tokenizer explicitly
model_output_dir = '/mnt/disks/disk1/results'
model.save_pretrained(model_output_dir)
tokenizer.save_pretrained(model_output_dir)
model.save_pretrained(model_output_dir)
tokenizer.save_pretrained(model_output_dir)

('/mnt/disks/disk1/results/tokenizer_config.json',
 '/mnt/disks/disk1/results/special_tokens_map.json',
 '/mnt/disks/disk1/results/vocab.json',
 '/mnt/disks/disk1/results/merges.txt',
 '/mnt/disks/disk1/results/added_tokens.json',
 '/mnt/disks/disk1/results/tokenizer.json')

In [42]:
def get_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

In [43]:
import re

def main(input_text):
    model_path = '/mnt/disks/disk1/results'
    model = AutoModelForCausalLM.from_pretrained(model_path)

    #Calculate the number of parameters in the model being used for inference
    num_params = get_model_parameters(model)
    #print(f"Number of parameters in the model: {num_params}")

    #Prepare input text for generation

    inputs = tokenizer(input_text, return_tensors="pt")

    #Generate text
    outputs = model.generate(**inputs, max_length = 500, num_return_sequences=1)

    #Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    #Extract only the response part by splitting based on "Response:"

    match = re.search(r"Response:\s*(.*)", generated_text)
    if match:
        response_text = match.group(1)

        #Remove extra spaces between sentences
        response_text = re.sub(r'\s{2,}', ' ', response_text)

        #Keep only up to the first sensible sentence-ending puntuation

        response_text = re.split(r'[.!?]', response_text)[0].strip() + '.'
        print("Response text:", response_text)
        return response_text
    else:
        return "No response found in the generated text."



In [44]:
# Example input for inference
example_input = """
(z0 / easy-05
    :ARG1 (z1 / scare-01
              :ARG1 (z2 / person
                        :ARG0-of (z3 / have-rel-role-91
                                     :ARG1 (z4 / i)
                                     :ARG2 (z5 / uncle))))
    :mod (z6 / certain))
"""
output = main(example_input)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response text: My uncle was definitely scared of me.


In [45]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(predicted_text, ground_truth_text):

  #Tokenize the texts into lists of words

  reference = [ground_truth_text.split()]
  hypothesis = predicted_text.split()
  if not hypothesis:
    return 0.0

  #Calc BLEU score with smoothing

  smoothie = SmoothingFunction().method4 #For short texts
  bleu_score = sentence_bleu(reference, hypothesis, smoothing_function=smoothie)

  return bleu_score


bleu_score = 0
valid_count = 0
k = 10

#Loop through dataset

for i in range(k):
  example_input = dataset_dict["test"][i]["amr_graph"]
  ground_truth_text = dataset_dict["test"][i]["answer"]

  #Tokenize and check input length
  tokenized_input = tokenizer(example_input, return_tensors='pt')

  input_length = tokenized_input['input_ids'].shape[1]


  if input_length >500:
    continue


  #Generate model output and calculate BLEU score
  model_output_text = main(example_input)
  bleu = calculate_bleu(model_output_text, ground_truth_text)

  # Only add BLEU score if it’s valid (greater than zero)
  if bleu > 0:
      bleu_score += bleu
      valid_count += 1  # Increment count of valid scores

# Calculate the average BLEU score only if there are valid scores
if valid_count > 0:
    avg_bleu_score = bleu_score / valid_count
else:
    avg_bleu_score = 0.0  # Set average to zero if no valid scores were found

print("Average BLEU score:", avg_bleu_score)





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response text: check your snowboard for visibility checks.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response text: The mafia thug was a designation in the street.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Response text: Sri Malyadri Balkrishnan has a temple in Nova Scotia.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response text: Muntaz al-Zaidi instead of living in the cave.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response text: The Final Cut has actors named kenny and kenny.
Response text: Chantal Delsol was born on April 16th, 1947.
Average BLEU score: 0.21879413844302556
